In [17]:
import pyproj
import math
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import ORBSLAMResults, helmert_transform, estimate_helmert_parameters, lm_estimate_transform

results = ORBSLAMResults("~/msc/shared_data/orbslam-out-utm-helmert-vabadusepst")

gps_trajectory_wgs = pd.DataFrame([(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                              for kf in results.keyframes[1:]], columns=['lat', 'lon', 'alt'])
slam_trajectory = np.array([(kf.x, kf.y, kf.z) for kf in results.keyframes[1:]])

# Create transformers for WGS84 <-> UTM35N
wgs2utm = pyproj.Transformer.from_crs(4326, 32635)
utm2wgs = pyproj.Transformer.from_crs(32635, 4326)

# Convert GPS trajectory (WGS84) to UTM35N
gps_trajectory_utm = np.array([wgs2utm.transform(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                                   for kf in results.keyframes[1:]])
# Find transformation between GPS and SLAM trajectories
helmert_params = estimate_helmert_parameters(slam_trajectory, gps_trajectory_utm)


# Apply transformation to SLAM trajectory
adjusted_slam_trajectory_utm, R, t, s = helmert_transform(helmert_params, slam_trajectory)
# Convert SLAM trajectory (UTM35N) to WGS84
adjusted_slam_trajectory_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2]) for p in adjusted_slam_trajectory_utm], columns=['lat', 'lon', 'alt'])

# Convert SLAM estimate (UTM35N) to WGS84
slam_estimate_wgs = pd.DataFrame([utm2wgs.transform(p.lat, p.lon, p.alt) for p in results.gps_estimate], columns=['lat', 'lon', 'alt'])

fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(lat=gps_trajectory_wgs['lat'], 
                     lon=gps_trajectory_wgs['lon'], 
                     mode='markers+lines',
                     marker=dict(color='blue'),
                     name='GPS'))
fig.add_trace(
    go.Scattermapbox(lat=adjusted_slam_trajectory_wgs['lat'], 
                     lon=adjusted_slam_trajectory_wgs['lon'], 
                     mode='markers+lines', 
                     marker=dict(color='red'),
                     name='SLAM'))
fig.add_trace(
    go.Scattermapbox(lat=slam_estimate_wgs['lat'], 
                     lon=slam_estimate_wgs['lon'], 
                     mode='markers+lines', 
                     marker=dict(color='forestgreen'),
                     name='SLAM estimate'))
fig.update_geos(projection_type="transverse mercator")
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=dict(lat=np.mean(gps_trajectory_wgs['lat']), lon=np.mean(gps_trajectory_wgs['lon'])), zoom=15),
                  margin={"t": 0, "b": 0, "l": 0, "r": 0}, 
                  height=800)
fig.show()